In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification, make_regression

In [ ]:
X_reg, y_reg = make_regression(n_samples=1000, n_features=10, noise=0.1)
X_clf, y_clf = make_classification(n_samples=1000, n_features=10, n_classes=2)

In [ ]:
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2)
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X_clf, y_clf, test_size=0.2)

In [ ]:
X_train_reg = torch.tensor(X_train_reg, dtype=torch.float32)
y_train_reg = torch.tensor(y_train_reg, dtype=torch.float32).view(-1, 1)
X_test_reg = torch.tensor(X_test_reg, dtype=torch.float32)
y_test_reg = torch.tensor(y_test_reg, dtype=torch.float32).view(-1, 1)

X_train_clf = torch.tensor(X_train_clf, dtype=torch.float32)
y_train_clf = torch.tensor(y_train_clf, dtype=torch.float32).view(-1, 1)
X_test_clf = torch.tensor(X_test_clf, dtype=torch.float32)
y_test_clf = torch.tensor(y_test_clf, dtype=torch.float32).view(-1, 1)

Linear Regression

In [ ]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim):
      super().__init__()
      self.linear = nn.Linear(input_dim, 1)
    def forward(self, x):
      return self.linear(x)

In [ ]:
model = LinearRegressionModel(X_train_reg.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)

In [ ]:
for epoch in range(10000):
    model.train()
    output = model(X_train_reg)
    loss = criterion(output, y_train_reg)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
preds = model(X_test_reg).detach().numpy()
print("Linear Regression MSE: ", mean_squared_error(y_test_reg.numpy(), preds))

Linear Regression MSE:  0.007592078298330307


Logistic Regression

In [ ]:
class LogisticRegressionModel(nn.Module):
  def __init__(self, input_dim):
    super().__init__()
    self.linear = nn.Linear(input_dim, 1)
  def forward(self, x):
    return torch.sigmoid(self.linear(x))

In [ ]:
model = LogisticRegressionModel(X_train_clf.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)

In [ ]:
for epoch in range(10000):
    model.train()
    output = model(X_train_clf)
    loss = criterion(output, y_train_clf)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
preds = model(X_test_clf).detach().numpy() > 0.5
print("Logistic Regression Accuracy:", accuracy_score(y_test_clf.numpy(), preds))

Logistic Regression Accuracy: 0.895


Neural Network Classifier

In [ ]:
class NeuralNetworkClassifier(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.f1 = nn.Linear(input_dim, 10)
        self.f2 = nn.Linear(10, 1)
    def forward(self, x):
        x = F.relu(self.f1(x))
        return F.sigmoid(self.f2(x))

In [ ]:
model = NeuralNetworkClassifier(X_train_clf.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)

In [ ]:
for epoch in range(10000):
    model.train()
    output = model(X_train_clf)
    loss = criterion(output, y_train_clf)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
preds = model(X_test_clf).detach().numpy() > 0.5
print("Simple Neural Network Accuracy:", accuracy_score(y_test_clf.numpy(), preds))

Simple Neural Network Accuracy: 0.87


Decision Tree Classifier

In [ ]:
tree = DecisionTreeClassifier()
tree.fit(X_train_clf.numpy(), y_train_clf.numpy().ravel())
preds = tree.predict(X_test_clf.numpy())
print("Decision Tree Accuracy:", accuracy_score(y_test_clf.numpy(), preds))

Decision Tree Accuracy: 0.875


Random Forest Classifer

In [ ]:
forest = RandomForestClassifier(n_estimators=100)
forest.fit(X_train_clf.numpy(), y_train_clf.numpy().ravel())
preds = forest.predict(X_test_clf.numpy())
print("Random Forest Accuracy:", accuracy_score(y_test_clf.numpy(), preds))

Random Forest Accuracy: 0.915


Deep Neural Network

In [ ]:
class DeepNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
model = DeepNN(X_train_clf.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(1000):
    model.train()
    output = model(X_train_clf)
    loss = criterion(output, y_train_clf)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
preds = model(X_test_clf).detach().numpy() > 0.5
print("Deep NN Accuracy:", accuracy_score(y_test_clf.numpy(), preds))

Deep NN Accuracy: 0.875
